# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 04**: Data Unions & Joins Pipeline

**Date**: September 23rd 2025

**Student Name**: Jaime Enrique Galindo Villegas

**Professor**: Pablo Camarillo Ramirez

In [32]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://9d7a5c0179a1:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [33]:
!ls /opt/spark/work-dir/data/car_service/

agencies  brands  cars	customers  rentals


In [34]:
# Build schema
# Import your module
from jaime_galindo.spark_utils import SparkUtils
from pyspark.sql.functions import get_json_object


In [35]:

# Generar esquema
schema_agencies = SparkUtils.generate_schema([("agency_id", "int"), ("agency_info", "string")])

# Generar df
df_agencies = spark.read.schema(schema_agencies).option("header", True).csv("/opt/spark/work-dir/data/car_service/agencies")

# Crear las nuevas columnas
df_agencies = df_agencies.withColumn("agency_name", get_json_object(df_agencies.agency_info, "$.agency_name")).drop("agency_info")


In [36]:
# Generar esquema
schema_brands = SparkUtils.generate_schema([("brand_id", "int"), ("brand_info", "string")])

# Generar df
df_brands = spark.read.schema(schema_brands).option("header", True).csv("/opt/spark/work-dir/data/car_service/brands")

# Crear las nuevas columnas
df_brands = df_brands.withColumn("brand_name", get_json_object(df_brands.brand_info, "$.brand_name")).drop("brand_info")


In [37]:
# Generar esquema
schema_cars = SparkUtils.generate_schema([("car_id", "int"), ("car_info", "string")])

# Generar df
df_cars = spark.read.schema(schema_cars).option("header", True).csv("/opt/spark/work-dir/data/car_service/cars")

# Crear las nuevas columnas
df_cars = df_cars.withColumn("car_name", get_json_object(df_cars.car_info, "$.car_name")).drop("car_info")


In [38]:
# Generar esquema
schema_customers = SparkUtils.generate_schema([("customer_id", "int"), ("customer_info", "string")])

# Generar df
df_customers = spark.read.schema(schema_customers).option("header", True).csv("/opt/spark/work-dir/data/car_service/customers/")

# Crear las nuevas columnas
df_customers = df_customers.withColumn("customer_name", get_json_object(df_customers.customer_info, "$.customer_name")).drop("customer_info")


In [39]:
# Generar esquema
schema_rentals = SparkUtils.generate_schema([("rental_id", "int"), ("rental_info", "string")])

# Generar df
df_rentals = spark.read.schema(schema_rentals).option("header", True).csv("/opt/spark/work-dir/data/car_service/rentals")

# Crear las nuevas columnas
df_rentals = df_rentals.withColumn("car_id", get_json_object(df_rentals.rental_info, "$.car_id"))
df_rentals = df_rentals.withColumn("customer_id", get_json_object(df_rentals.rental_info, "$.customer_id"))
df_rentals = df_rentals.withColumn("agency_id", get_json_object(df_rentals.rental_info, "$.agency_id"))
df_rentals = df_rentals.drop("rental_info")


In [ ]:
# Joins
df_rentals = df_rentals.join(df_cars, on="car_id", how="left")
df_rentals = df_rentals.join(df_customers, on="customer_id", how="left")
df_rentals = df_rentals.join(df_agencies, on="agency_id", how="left")

# Eliminar columnas extra
df_rentals = df_rentals.drop("agency_id", "customer_id", "car_id")

df_rentals.show(truncate=False)

+---------+-----------------------------------+---------------+-------------+
|rental_id|car_name                           |customer_name  |agency_name  |
+---------+-----------------------------------+---------------+-------------+
|11891    |Wallace-Carlson Model 9            |Margaret Jones |NYC Rentals  |
|11892    |Grimes-Green Model 8               |Albert Williams|LA Car Rental|
|11893    |Stewart-Allen Model 5              |Caleb Fleming  |SF Cars      |
|11894    |Campos PLC Model 4                 |Andrew Butler  |NYC Rentals  |
|11895    |Wagner LLC Model 1                 |Kristin Potts  |SF Cars      |
|11896    |Jones, Jefferson and Rivera Model 7|Jeremy Parks   |LA Car Rental|
|11897    |Lopez and Sons Model 9             |Terry Wells    |Zapopan Auto |
|11898    |Salazar Ltd Model 8                |Marc Williams  |SF Cars      |
|11899    |Villanueva PLC Model 7             |Danny Williams |LA Car Rental|
|11900    |Faulkner-Howard Model 5            |Eric Owens PhD |S

In [41]:
# sc.stop()
